# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
        
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creates a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Sets KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

# The primary key comprises of two fields: session_id is the partition key and item_in_session is the clustering key.
# The partitioning of data is done by session_id and within that partitioning, rows are ordered by item_in_session
queryA = "CREATE TABLE IF NOT EXISTS music_history (session_id int, item_in_session int, artist text, song text, length double,\
                                                    PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(queryA)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigns the INSERT statements into the `query` variable
        query = "INSERT INTO music_history (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Assigns which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Solution to the first query
query1 = "SELECT artist, song, length from music_history WHERE session_id=88 AND item_in_session=1"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Cat Stevens Wild World 200.202


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

# The primary key comprises of three fields: a two field partition key (user_id, session_id) and a clustering key (item_in_session).
# The partitioning of data is done user_id and session_id and within this partitioning, the rows are ordered by item_in_session.
queryB = "CREATE TABLE IF NOT EXISTS artist_and_song_in_user_session (user_id int, session_id int, item_in_session int, artist text, song text, first_name text,\
                                                    last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"

try:
    session.execute(queryB)
except Exception as e:
    print(e)
    
# insert data into second_table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigns the INSERT statements into the `query` variable
        query = "INSERT INTO artist_and_song_in_user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assigns which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# Solution to the second query
query2 = "SELECT artist, song, first_name, last_name FROM artist_and_song_in_user_session WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [12]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# The primary key is comprised of two fields: a partition key (song) and a clustering key (user_id).
# The partitioning of data is done by the song title and within this partitioning the rows are ordered by the user_id.
queryC = "CREATE TABLE IF NOT EXISTS who_listend_to_what (song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"

try:
    session.execute(queryC)
except Exception as e:
    print(e)
    
# insert data into second_table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assigns the INSERT statements into the `query` variable
        query = "INSERT INTO who_listend_to_what (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assigns which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# Solution to the third query
query3 = "SELECT first_name, last_name FROM who_listend_to_what WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
# Drop the table before closing out the sessions
query = "drop table music_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table artist_and_song_in_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table who_listend_to_what"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()